# Creating an American Football Reinforcement Learning Enviornment 

In [1]:
# Package dependencies
import pygame
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import pandas as pd
import nfl_data_py as nfl

pygame 2.5.2 (SDL 2.28.3, Python 3.8.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Custom dependencies
from FootballPlay import FootballPlay
from PreprocessTrackingData import PreprocessTrackingData

In [3]:
# Turn off pandas column limits
pd.set_option('display.max_columns', None)

In [4]:
# Read in the data
tracking = pd.read_csv("data/tracking_week_4.csv")
plays = pd.read_csv("data/plays.csv")
players = pd.read_csv("data/players.csv")
colors = pd.read_csv("data/colors.csv")

In [5]:
# Select game and play
gameId = 2022092900
playId = 57
nflId = 43321

In [6]:
tracking[(tracking["gameId"] == gameId) & (tracking["playId"] == playId) & (tracking['club'] == 'MIA')]

,gameId,playId,nflId,displayName,frameId,time,jerseyNumber,club,playDirection,x,y,s,a,dis,o,dir,event
49,2022092900,57,43321.0,Emmanuel Ogbah,1,2022-09-29 20:16:00.099999,91.0,MIA,left,84.17,25.16,0.01,0.01,0.00,91.16,187.64,NaN
50,2022092900,57,43321.0,Emmanuel Ogbah,2,2022-09-29 20:16:00.200000,91.0,MIA,left,84.17,25.16,0.01,0.01,0.00,91.16,187.57,NaN
51,2022092900,57,43321.0,Emmanuel Ogbah,3,2022-09-29 20:16:00.299999,91.0,MIA,left,84.17,25.16,0.01,0.01,0.00,91.85,187.61,NaN
52,2022092900,57,43321.0,Emmanuel Ogbah,4,2022-09-29 20:16:00.400000,91.0,MIA,left,84.17,25.16,0.01,0.01,0.01,92.99,187.43,NaN
53,2022092900,57,43321.0,Emmanuel Ogbah,5,2022-09-29 20:16:00.500000,91.0,MIA,left,84.17,25.15,0.03,0.20,0.01,92.99,142.35,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1073,2022092900,57,54936.0,Kader Kohou,45,2022-09-29 20:16:04.500000,28.0,MIA,left,80.00,17.25,3.77,1.26,0.38,5.15,359.31,tackle
1074,2022092900,57,54936.0,Kader Kohou,46,2022-09-29 20:16:04.599999,28.0,MIA,left,80.00,17.62,3.63,1.38,0.37,5.15,359.85,NaN
1075,2022092900,57,54936.0,Kader Kohou,47,2022-09-29 20:16:04.700000,28.0,MIA,left,80.00,17.97,3.50,1.41,0.36,5.15,0.45,NaN
1076,2022092900,57,54936.0,Kader Kohou,48,2022-09-29 20:16:04.799999,28.0,MIA,left,80.00,18.32,3.37,1.43,0.34,4.52,359.30,NaN


In [7]:
# Define preprocessor
preprocessor = PreprocessTrackingData(tracking, 
                                      plays,
                                      players,
                                      colors,
                                      gameId, 
                                      playId
                                      )

In [8]:
# Get processed data
play_data = preprocessor.get_processed_data()

# Find the maximum frame ID
max_frames = play_data['frameId'].max()

In [9]:
# Define the environment
env = FootballPlay(play_data, nflId, render_mode='human', max_frames=max_frames)

In [10]:
for _ in range(1000):
    observation, info = env.reset()
    done = False

    while not done:
        action = env.action_space.sample()  # Sample an action
        observation, reward, terminated, truncated, info = env.step(action)

        if terminated or truncated:
            done = True  # Mark the episode as done

env.close()

: 